# SVHN/CIFAR-10 OOD Experiment
> Can we get better by training on our assumptions?

In [ ]:
# default_exp svhn_cifar10_ood_experiment

In [ ]:
# hide
import blackhc.project.script

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback
from dataclasses import dataclass
from typing import Type, Union

import torch
import torch.utils.data
from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments
from torch.utils.data import Dataset

import batchbald_redux.acquisition_functions as acquisition_functions
from batchbald_redux.acquisition_functions import (
    CandidateBatchComputer,
    EvalCandidateBatchComputer,
)
from batchbald_redux.active_learning import ActiveLearningData, RandomFixedLengthSampler
from batchbald_redux.black_box_model_training import evaluate, train
from batchbald_redux.dataset_challenges import (
    AdditiveGaussianNoise,
    AliasDataset,
    NamedDataset,
    create_named_mnist,
    get_balanced_sample_indices_by_class,
    get_base_dataset_index,
    get_target,
)
from batchbald_redux.datasets import get_dataset
from batchbald_redux.datasets import train_validation_split
from batchbald_redux.di import DependencyInjection
from batchbald_redux.fast_mnist import FastFashionMNIST, FastMNIST
from batchbald_redux.model_optimizer_factory import ModelOptimizerFactory
from batchbald_redux.resnet_models import Cifar10BayesianResnetFactory
from batchbald_redux.train_eval_model import (
    TrainEvalModel,
    TrainSelfDistillationEvalModel,
)
from batchbald_redux.trained_model import TrainedMCDropoutModel

In [ ]:
# exports


@dataclass
class ExperimentData:
    active_learning: ActiveLearningData
    augmentation_node: AliasDataset
    unaugmented_train_dataset: Dataset
    augmented_train_dataset: Dataset
    ood_dataset: NamedDataset
    validation_dataset: Dataset
    test_dataset: Dataset
    evaluation_dataset: Dataset
    initial_training_set_indices: [int]
    evaluation_set_indices: [int]
        
    def toggle_augmentations(self, augment: bool):
        if augment:
            self.augmentation_node.dataset = self.augmented_train_dataset
        else:
            self.augmentation_node.dataset = self.unaugmented_train_dataset


@dataclass
class ExperimentDataConfig:
    uniform_ood: bool
    id_dataset_name: str
    ood_dataset_name: str

    initial_training_set_size: int
    validation_set_size: int
    evaluation_set_size: int
    id_repetitions: float
    ood_repetitions: float
    add_dataset_noise: bool
    validation_split_random_state: int

    device: str

    def load(self) -> ExperimentData:
        return load_experiment_data(
            id_dataset_name=self.id_dataset_name,
            ood_dataset_name=self.ood_dataset_name,
            initial_training_set_size=self.initial_training_set_size,
            validation_set_size=self.validation_set_size,
            evaluation_set_size=self.evaluation_set_size,
            id_repetitions=self.id_repetitions,
            ood_repetitions=self.ood_repetitions,
            add_dataset_noise=self.add_dataset_noise,
            uniform_ood=self.uniform_ood,
            validation_split_random_state=self.validation_split_random_state,
            device=self.device,
        )


def load_experiment_data(
    *,
    id_dataset_name: str,
    ood_dataset_name: str,
    initial_training_set_size: int,
    validation_set_size: int,
    evaluation_set_size: int,
    id_repetitions: float,
    ood_repetitions: float,
    add_dataset_noise: bool,
    validation_split_random_state: int,
    device: str,
    uniform_ood: bool,
) -> ExperimentData:    
    split_dataset = get_dataset(id_dataset_name, root="data", train_augmentation=True, validation_set_size=validation_set_size,
                                validation_split_random_state=validation_split_random_state, normalize_like_cifar10=True)
    unaugmented_split_dataset = get_dataset(id_dataset_name, root="data", train_augmentation=True, validation_set_size=validation_set_size,
                                validation_split_random_state=validation_split_random_state, normalize_like_cifar10=True)
    
    augmentation_node = AliasDataset(split_dataset.train, "Augmentation Node")
    train_dataset=augmentation_node
    
    # If we reduce the train set, we need to do so before picking the initial train set.
    if id_repetitions < 1:
        train_dataset = train_dataset * id_repetitions

    num_classes = train_dataset.get_num_classes()
    initial_samples_per_class = initial_training_set_size // num_classes
    evaluation_set_samples_per_class = evaluation_set_size // num_classes
    samples_per_class = initial_samples_per_class + evaluation_set_samples_per_class
    balanced_samples_indices = get_balanced_sample_indices_by_class(
        train_dataset,
        num_classes=num_classes,
        samples_per_class=samples_per_class,
        seed=validation_split_random_state,
    )

    initial_training_set_indices = [
        idx for by_class in balanced_samples_indices.values() for idx in by_class[:initial_samples_per_class]
    ]
    evaluation_set_indices = [
        idx for by_class in balanced_samples_indices.values() for idx in by_class[initial_samples_per_class:]
    ]

    # If we over-sample the train set, we do so after picking the initial train set to avoid duplicates.
    if id_repetitions > 1:
        train_dataset = train_dataset * id_repetitions

    original_ood_dataset = get_dataset(ood_dataset_name, root="data", train_augmentation=False, normalize_like_cifar10=True).train
    if uniform_ood:
        train_dataset = train_dataset.one_hot(device=device)
        ood_dataset = original_ood_dataset.uniform_target(device=device)
    else:
        ood_dataset = original_ood_dataset.constant_target(
            target=torch.tensor(-1, device=device), num_classes=train_dataset.get_num_classes()
        )

    if ood_repetitions != 1:
        ood_dataset = ood_dataset * ood_repetitions

    train_dataset = train_dataset + ood_dataset

    if add_dataset_noise:
        train_dataset = AdditiveGaussianNoise(train_dataset, 0.1)

    active_learning_data = ActiveLearningData(train_dataset)

    active_learning_data.acquire_base_indices(initial_training_set_indices)

    evaluation_dataset = AliasDataset(
        active_learning_data.extract_dataset_from_base_indices(evaluation_set_indices),
        f"Evaluation Set ({len(evaluation_set_indices)} samples)",
    )

    return ExperimentData(
        active_learning=active_learning_data,
        augmentation_node=augmentation_node,
        augmented_train_dataset=split_dataset.train,
        unaugmented_train_dataset=unaugmented_split_dataset.train,
        ood_dataset=original_ood_dataset,
        validation_dataset=split_dataset.validation,
        test_dataset=split_dataset.test,
        evaluation_dataset=evaluation_dataset,
        initial_training_set_indices=initial_training_set_indices,
        evaluation_set_indices=evaluation_set_indices,
    )

In [ ]:
# exports


@dataclass
class OodExperiment:
    seed: int
    uniform_ood: bool

    id_dataset_name: str
    ood_dataset_name: str
    initial_training_set_size: int = 0
    validation_set_size: int = 1024
    evaluation_set_size: int = 1024
    id_repetitions: float = 1
    ood_repetitions: float = 1
    add_dataset_noise: bool = False
    validation_split_random_state: int = 0

    acquisition_size: int = 5
    max_training_set: int = 200
    num_pool_samples: int = 20
    num_validation_samples: int = 20
    num_training_samples: int = 1
    num_patience_epochs: int = 3
    max_training_epochs: int = 60
    training_batch_size: int = 64
    device: str = "cuda"
    min_samples_per_epoch: int = 5056
    acquisition_function: Union[
        Type[CandidateBatchComputer], Type[EvalCandidateBatchComputer]
    ] = acquisition_functions.BALD
    train_eval_model: Type[TrainEvalModel] = TrainSelfDistillationEvalModel
    model_optimizer_factory: Type[ModelOptimizerFactory] = Cifar10BayesianResnetFactory
    acquisition_function_args: dict = None
    temperature: float = 0.0

    def load_experiment_data(self) -> ExperimentData:
        di = DependencyInjection(vars(self))
        edc: ExperimentDataConfig = di.create_dataclass_type(ExperimentDataConfig)
        return edc.load()

    # Simple Dependency Injection
    def create_acquisition_function(self):
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.acquisition_function)

    def create_train_eval_model(self, runtime_config) -> TrainEvalModel:
        config = {**vars(self), **runtime_config}
        di = DependencyInjection(config, [])
        return di.create_dataclass_type(self.train_eval_model)

    def run(self, store):
        torch.manual_seed(self.seed)

        # Active Learning setup
        data = self.load_experiment_data()
        store["dataset_info"] = dict(training=repr(data.active_learning.base_dataset), test=repr(data.test_dataset))
        store["initial_training_set_indices"] = data.initial_training_set_indices
        store["evaluation_set_indices"] = data.evaluation_set_indices

        train_loader = torch.utils.data.DataLoader(
            data.active_learning.training_dataset,
            batch_size=self.training_batch_size,
            sampler=RandomFixedLengthSampler(data.active_learning.training_dataset, self.min_samples_per_epoch),
            drop_last=True,
        )
        pool_loader = torch.utils.data.DataLoader(
            data.active_learning.pool_dataset, batch_size=128, drop_last=False, shuffle=False
        )

        validation_loader = torch.utils.data.DataLoader(data.validation_dataset, batch_size=512, drop_last=False)
        test_loader = torch.utils.data.DataLoader(data.test_dataset, batch_size=512, drop_last=False)

        store["active_learning_steps"] = []
        active_learning_steps = store["active_learning_steps"]

        acquisition_function = self.create_acquisition_function()

        num_iterations = 0
        max_iterations = int(1.5 * (self.max_training_set - self.initial_training_set_size) / self.acquisition_size)

        # Active Training Loop
        while True:
            training_set_size = len(data.active_learning.training_dataset)
            print(f"Training set size {training_set_size}:")

            # iteration_log = dict(training={}, pool_training={}, evaluation_metrics=None, acquisition=None)
            active_learning_steps.append({})
            iteration_log = active_learning_steps[-1]

            iteration_log["training"] = {}

            model_optimizer = self.model_optimizer_factory().create_model_optimizer()

            if self.uniform_ood:
                loss = torch.nn.KLDivLoss(log_target=False, reduction="batchmean")
                validation_loss = torch.nn.NLLLoss()
            else:
                loss = validation_loss = torch.nn.NLLLoss()
                
            data.toggle_augmentations(True)

            train(
                model=model_optimizer.model,
                optimizer=model_optimizer.optimizer,
                training_samples=self.num_training_samples,
                validation_samples=self.num_validation_samples,
                train_loader=train_loader,
                validation_loader=validation_loader,
                patience=self.num_patience_epochs,
                max_epochs=self.max_training_epochs,
                device=self.device,
                training_log=iteration_log["training"],
                loss=loss,
                validation_loss=validation_loss,
            )
            
            data.toggle_augmentations(False)

            evaluation_metrics = evaluate(
                model=model_optimizer.model,
                num_samples=self.num_validation_samples,
                loader=test_loader,
                device=self.device,
            )
            iteration_log["evaluation_metrics"] = evaluation_metrics
            print(f"Perf after training {evaluation_metrics}")

            if training_set_size >= self.max_training_set or num_iterations >= max_iterations:
                print("Done.")
                break

            trained_model = TrainedMCDropoutModel(num_samples=self.num_pool_samples, model=model_optimizer.model)

            if isinstance(acquisition_function, CandidateBatchComputer):
                candidate_batch = acquisition_function.compute_candidate_batch(trained_model, pool_loader, self.device)
            elif isinstance(acquisition_function, EvalCandidateBatchComputer):
                current_max_epochs = iteration_log["training"]["best_epoch"]

                if self.evaluation_set_size:
                    eval_dataset = data.evaluation_dataset
                else:
                    eval_dataset = data.active_learning.pool_dataset

                train_eval_model = self.create_train_eval_model(
                    dict(
                        max_epochs=current_max_epochs + 2,
                        training_dataset=data.active_learning.training_dataset,
                        eval_dataset=eval_dataset,
                        validation_loader=validation_loader,
                        trained_model=trained_model,
                        data=data,
                    )
                )

                iteration_log["eval_training"] = {}
                trained_eval_model = train_eval_model(training_log=iteration_log["eval_training"], device=self.device)

                candidate_batch = acquisition_function.compute_candidate_batch(
                    trained_model, trained_eval_model, pool_loader, device=self.device
                )
            else:
                raise ValueError(f"Unknown acquisition function {acquisition_function}!")

            candidate_global_dataset_indices = []
            candidate_labels = []
            for index in candidate_batch.indices:
                base_di = get_base_dataset_index(data.active_learning.pool_dataset, index)
                dataset_type = "ood" if base_di.dataset == data.ood_dataset else "id"
                candidate_global_dataset_indices.append((dataset_type, base_di.index))
                label = get_target(data.active_learning.pool_dataset, index)
                candidate_labels.append(label)

            iteration_log["acquisition"] = dict(
                indices=candidate_global_dataset_indices, labels=candidate_labels, scores=candidate_batch.scores
            )

            print(candidate_batch)
            print(candidate_global_dataset_indices)

            if self.uniform_ood:
                data.active_learning.acquire(candidate_batch.indices)
            else:
                data.active_learning.acquire(
                    [index for index, label in zip(candidate_batch.indices, candidate_labels) if label != -1]
                )

            ls = ", ".join(f"{label} ({score:.4})" for label, score in zip(candidate_labels, candidate_batch.scores))
            print(f"Acquiring (label, score)s: {ls}")

            num_iterations += 1

In [ ]:
# experiment

experiment = OodExperiment(
    uniform_ood=True,
    id_dataset_name="CIFAR-10",
    ood_dataset_name="SVHN",
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: ExperimentDataConfig(
	uniform_ood=True,
	id_dataset_name=CIFAR-10,
	ood_dataset_name=SVHN,
	initial_training_set_size=20,
	validation_set_size=1024,
	evaluation_set_size=100,
	id_repetitions=1,
	ood_repetitions=1,
	add_dataset_noise=False,
	validation_split_random_state=0,
	device=cuda
)
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: datadata/SVHN/train_32x32.mat
Using downloaded and verified file: datadata/SVHN/train_32x32.mat
Using downloaded and verified file: datadata/SVHN/test_32x32.mat
Creating: EvalBALD(
	acquisition_size=10
)
Training set size 20:


100%|##########| 1/1 [00:00<?, ?it/s]

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.162109375, 'crossentropy': 8.790198802947998}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.162109375)
RestoringEarlyStopping: Restoring optimizer.


[1/20]   5%|5          [00:00<?]

...

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.1845703125, 'crossentropy': 9.042139053344727}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.1845703125)
RestoringEarlyStopping: Restoring optimizer.


[1/20]   5%|5          [00:00<?]

Perf after training {'accuracy': 0.1822, 'crossentropy': 9.2006466796875}
Done.


{'dataset_info': {'training': "(Augmentation Node | one_hot_targets{'num_classes': 10}) + ('SVHN (Train, seed=0, 73257 samples)' | uniform_targets{'num_classes': 10})",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [12980,
  44617,
  6984,
  21168,
  33976,
  35571,
  33058,
  43729,
  26944,
  24745,
  66,
  14046,
  46542,
  39478,
  6000,
  5915,
  39360,
  20774,
  27084,
  44464],
 'evaluation_set_indices': [3812,
  42704,
  6729,
  38942,
  48125,
  16968,
  5652,
  4045,
  10740,
  19606,
  37164,
  33354,
  47307,
  17878,
  26665,
  40819,
  14805,
  201,
  47956,
  44739,
  15578,
  36667,
  5551,
  23088,
  32496,
  5705,
  23255,
  25559,
  11975,
  44032,
  47518,
  36303,
  18452,
  34447,
  24821,
  36157,
  48089,
  25120,
  44689,
  6509,
  11001,
  6995,
  10899,
  36881,
  7002,
  19049,
  13388,
  40737,
  9210,
  22684,
  45656,
  5604,
  9134,
  35979,
  19757,
  43627,
  35248,
  23566,
  727,
  34909,
  25443,
  45862,
  30730,
  9611,
  43077

In [ ]:
# experiment

print(
    OodExperiment(seed=0, device="cpu", id_dataset_name="SVHN", ood_dataset_name="CIFAR-10", uniform_ood=False)
    .load_experiment_data()
    .active_learning.base_dataset
)
print(
    OodExperiment(seed=1, device="cpu", id_dataset_name="CIFAR-10", ood_dataset_name="SVHN", uniform_ood=True)
    .load_experiment_data()
    .active_learning.base_dataset
)

Creating: ExperimentDataConfig(
	uniform_ood=False,
	id_dataset_name=SVHN,
	ood_dataset_name=CIFAR-10,
	initial_training_set_size=0,
	validation_set_size=1024,
	evaluation_set_size=1024,
	id_repetitions=1,
	ood_repetitions=1,
	add_dataset_noise=False,
	validation_split_random_state=0,
	device=cpu
)
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/test_32x32.mat
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
(Augmentation Node) + ('CIFAR-10 (Train, seed=0, 50000 samples)' | constant_target{'target': tensor(-1), 'num_classes': 10})
Creating: ExperimentDataConfig(
	uniform_ood=True,
	id_dataset_name=CIFAR-10,
	ood_dataset_

In [ ]:
results

{'dataset_info': {'training': "(Augmentation Node | one_hot_targets{'num_classes': 10}) + ('SVHN (Train, seed=0, 73257 samples)' | uniform_targets{'num_classes': 10})",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [12980,
  44617,
  6984,
  21168,
  33976,
  35571,
  33058,
  43729,
  26944,
  24745,
  66,
  14046,
  46542,
  39478,
  6000,
  5915,
  39360,
  20774,
  27084,
  44464],
 'evaluation_set_indices': [3812,
  42704,
  6729,
  38942,
  48125,
  16968,
  5652,
  4045,
  10740,
  19606,
  37164,
  33354,
  47307,
  17878,
  26665,
  40819,
  14805,
  201,
  47956,
  44739,
  15578,
  36667,
  5551,
  23088,
  32496,
  5705,
  23255,
  25559,
  11975,
  44032,
  47518,
  36303,
  18452,
  34447,
  24821,
  36157,
  48089,
  25120,
  44689,
  6509,
  11001,
  6995,
  10899,
  36881,
  7002,
  19049,
  13388,
  40737,
  9210,
  22684,
  45656,
  5604,
  9134,
  35979,
  19757,
  43627,
  35248,
  23566,
  727,
  34909,
  25443,
  45862,
  30730,
  9611,
  43077

In [ ]:
# experiment
experiment = OodExperiment(
    uniform_ood=False,
    id_dataset_type=FastMNIST,
    ood_dataset=FastFashionMNIST,
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.BALD,
    evaluation_set_size=100,
    acquisition_size=10,
    num_pool_samples=2,
    temperature=5,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: ExperimentDataConfig(
	uniform_ood=False,
	id_dataset_type=<class 'batchbald_redux.fast_mnist.FastMNIST'>,
	ood_dataset_type=<class 'batchbald_redux.fast_mnist.FastFashionMNIST'>,
	initial_training_set_size=20,
	validation_set_size=1024,
	evaluation_set_size=100,
	id_repetitions=1,
	ood_repetitions=1,
	add_dataset_noise=False,
	validation_split_random_state=0,
	device=cuda
)
Creating: BALD(
	acquisition_size=10
)
Training set size 20:


100%|##########| 1/1 [00:00<?, ?it/s]

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.6328125, 'crossentropy': 2.2057557106018066}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.6328125)
RestoringEarlyStopping: Restoring optimizer.


[1/20]   5%|5          [00:00<?]

...

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.611328125, 'crossentropy': 2.4593361616134644}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.611328125)
RestoringEarlyStopping: Restoring optimizer.


[1/20]   5%|5          [00:00<?]

Perf after training {'accuracy': 0.6549, 'crossentropy': 2.1096548225402834}
Done.


{'dataset_info': {'training': "(FastMNIST (train; 58976 samples)) + ('FastFashionMNIST Train (60000 samples)' | constant_target{'target': tensor(-1, device='cuda:0'), 'num_classes': 10})",
  'test': "'FastMNIST Test (10000 samples)'"},
 'initial_training_set_indices': [53434,
  8533,
  14640,
  39579,
  30392,
  58125,
  37915,
  3091,
  57520,
  43803,
  44119,
  52296,
  58226,
  40334,
  46037,
  22015,
  22304,
  43812,
  12640,
  53689],
 'evaluation_set_indices': [29974,
  55573,
  35472,
  44048,
  48031,
  5616,
  10110,
  47420,
  56990,
  34198,
  3792,
  5715,
  15969,
  32775,
  19757,
  34588,
  28991,
  47417,
  26501,
  12108,
  5573,
  48032,
  40646,
  43252,
  2404,
  36797,
  29079,
  40018,
  37047,
  41512,
  45567,
  801,
  10664,
  52801,
  42890,
  32972,
  45974,
  20801,
  23496,
  5803,
  10508,
  46870,
  49549,
  306,
  38725,
  13074,
  19689,
  27135,
  16068,
  18137,
  2728,
  43321,
  29950,
  380,
  27254,
  50466,
  31965,
  24052,
  44454,
  20076,


In [ ]:
# experiment

experiment = OodExperiment(
    uniform_ood=False,
    id_dataset_type=FastMNIST,
    ood_dataset=FastFashionMNIST,
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    num_pool_samples=2,
    temperature=5,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: ExperimentDataConfig(
	uniform_ood=False,
	id_dataset_type=<class 'batchbald_redux.fast_mnist.FastMNIST'>,
	ood_dataset_type=<class 'batchbald_redux.fast_mnist.FastFashionMNIST'>,
	initial_training_set_size=20,
	validation_set_size=1024,
	evaluation_set_size=100,
	id_repetitions=1,
	ood_repetitions=1,
	add_dataset_noise=False,
	validation_split_random_state=0,
	device=cuda
)
Creating: EvalBALD(
	acquisition_size=10
)
Training set size 20:


100%|##########| 1/1 [00:00<?, ?it/s]

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.6328125, 'crossentropy': 2.1983988285064697}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.6328125)
RestoringEarlyStopping: Restoring optimizer.


[1/20]   5%|5          [00:00<?]

...

[1/79]   1%|1          [00:00<?]

Epoch metrics: {'accuracy': 0.6474609375, 'crossentropy': 2.016913414001465}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.6474609375)
RestoringEarlyStopping: Restoring optimizer.


[1/20]   5%|5          [00:00<?]

Perf after training {'accuracy': 0.6889, 'crossentropy': 1.9067923236846924}
Done.


{'dataset_info': {'training': "(FastMNIST (train; 58976 samples)) + ('FastFashionMNIST Train (60000 samples)' | constant_target{'target': tensor(-1, device='cuda:0'), 'num_classes': 10})",
  'test': "'FastMNIST Test (10000 samples)'"},
 'initial_training_set_indices': [53434,
  8533,
  14640,
  39579,
  30392,
  58125,
  37915,
  3091,
  57520,
  43803,
  44119,
  52296,
  58226,
  40334,
  46037,
  22015,
  22304,
  43812,
  12640,
  53689],
 'evaluation_set_indices': [29974,
  55573,
  35472,
  44048,
  48031,
  5616,
  10110,
  47420,
  56990,
  34198,
  3792,
  5715,
  15969,
  32775,
  19757,
  34588,
  28991,
  47417,
  26501,
  12108,
  5573,
  48032,
  40646,
  43252,
  2404,
  36797,
  29079,
  40018,
  37047,
  41512,
  45567,
  801,
  10664,
  52801,
  42890,
  32972,
  45974,
  20801,
  23496,
  5803,
  10508,
  46870,
  49549,
  306,
  38725,
  13074,
  19689,
  27135,
  16068,
  18137,
  2728,
  43321,
  29950,
  380,
  27254,
  50466,
  31965,
  24052,
  44454,
  20076,


In [ ]:
# exports

configs = [
    OodExperiment(
        seed=seed + 1234,
        uniform_ood=uniform_ood,
        acquisition_function=acquisition_function,
        acquisition_size=5,
        num_pool_samples=num_pool_samples,
        evaluation_set_size=evaluation_set_size,
        id_dataset_name=id_dataset_name,
        ood_dataset_name=ood_dataset_name,
    )
    for seed in range(3)
    for acquisition_function in [acquisition_functions.BatchEvalBALD, acquisition_functions.BatchBALD]
    for evaluation_set_size in [1024]
    for num_pool_samples in [100]
    for uniform_ood in [True, False]
    for id_dataset_name, ood_dataset_name in [("CIFAR-10", "SVHN"), ("SVHN", "CIFAR-10")]
]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
len(configs)

24

In [ ]:
# slow
import prettyprinter

prettyprinter.install_extras(include={"dataclasses"})

prettyprinter.pprint(configs)

[
    OodExperiment(
        seed=1234,
        uniform_ood=True,
        id_dataset_name='CIFAR-10',
        ood_dataset_name='SVHN',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=True,
        id_dataset_name='SVHN',
        ood_dataset_name='CIFAR-10',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=False,
        id_dataset_name='CIFAR-10',
        ood_dataset_name='SVHN',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=False,
        id_dataset_name='SVHN',
        ood_dataset_name='CIFAR-10',
        num_pool_samples=100,
        # class
        acquisit